In [2]:
#default import 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import sqrt
# learners
from sklearn.datasets import make_blobs
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
#
from sklearn.model_selection import train_test_split 


In [3]:

# 使用するモデルの定義
def get_models():
    models = list()
    models.append(GradientBoostingClassifier())
    models.append(LogisticRegression(solver='liblinear'))
#     models.append(DecisionTreeClassifier())
    models.append(SVC(gamma='scale', probability=True))
    models.append(GaussianNB())
    models.append(KNeighborsClassifier())
    models.append(AdaBoostClassifier())
    models.append(BaggingClassifier(n_estimators=10))
    models.append(RandomForestClassifier(n_estimators=10))
    models.append(ExtraTreesClassifier(n_estimators=10))
    return models


# super learner に用いる特徴量の作成
def get_out_of_fold_predictions(X, y, models):
    meta_X, meta_y = list(),list()
    kfold = KFold(n_splits=10,shuffle =True)
    for train_ix,test_ix in kfold.split(X):
        
        fold_yhats = list()

        train_X, test_X = X[train_ix],X[test_ix]
        train_y, test_y = y[train_ix],y[test_ix]
        meta_y.extend(test_y)#ここよくわからん
#         print(len(test_X))
        
        for model in models:
            model.fit(train_X,train_y)
            yhat = model.predict(test_X)
            
            fold_yhats.append(yhat.reshape(len(yhat),1))
        meta_X.append(np.hstack(fold_yhats))
    return np.vstack(meta_X),np.asarray(meta_y)

# 特徴量として使うモデルの集合
def fit_base_models(X, y, models):
    for model in models:
        model.fit(X, y)
# 最終的に予測するモデル
def fit_meta_model(X, y):
    model = LogisticRegression(solver = 'liblinear')
#     model = GradientBoostingClassifier()
    model.fit(X, y)
    return model

# 評価用関数
def evaluate_models(X, y, models):
    for model in models:
        yhat = model.predict(X)
        mse = mean_squared_error(y, yhat)
        print('%s: RMSE %.3f' % (model.__class__.__name__, sqrt(mse)))
# super learner による予測
def super_learner_predictions(X,models,meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict(X)
        meta_X.append(yhat.reshape(len(yhat),1))
    meta_X = np.hstack(meta_X)
    
    return meta_model.predict(meta_X)

In [4]:
data = pd.read_csv('../data/ObsData.csv')
chspred = pd.read_csv('../data/chspred.csv')
washb = pd.read_csv("../data/convert_wash.csv")

In [6]:
washb

,whz,tr,fracode,month,aged,sex,momage,momedu,momheight,hfiacat,...,asset_table,asset_chair,asset_khat,asset_chouki,asset_tv,asset_refrig,asset_bike,asset_moto,asset_sewmach,asset_mobile
0,0.00,Control,N05265,9,268,2,30.0,Primary (1-5y),146.40,Food Secure,...,1,1,1,0,1,0,0,0,0,1
1,-1.16,Control,N05265,9,286,2,25.0,Primary (1-5y),148.75,Moderately Food Insecure,...,1,0,1,1,0,0,0,0,0,1
2,-1.05,Control,N08002,9,264,2,25.0,Primary (1-5y),152.15,Food Secure,...,0,1,0,1,0,0,0,0,0,1
3,-1.26,Control,N08002,9,252,1,28.0,Primary (1-5y),140.25,Food Secure,...,1,1,1,0,0,0,1,0,0,1
4,-0.59,Control,N06531,9,336,1,19.0,Secondary (>5y),150.95,Food Secure,...,1,1,1,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4690,-0.74,Nutrition + WSH,N04681,9,294,1,30.0,Secondary (>5y),146.55,Food Secure,...,1,1,1,1,0,0,1,0,0,1
4691,0.02,Nutrition + WSH,N04681,9,348,2,18.0,Secondary (>5y),154.70,Food Secure,...,1,1,1,1,1,1,0,0,0,1
4692,-0.96,Nutrition + WSH,N04681,9,278,1,22.0,Secondary (>5y),147.10,Food Secure,...,0,0,1,0,0,0,0,0,0,1
4693,-1.11,Nutrition + WSH,N08002,9,357,1,23.0,Primary (1-5y),144.70,Food Secure,...,0,1,1,0,1,0,1,0,0,1
